In [ ]:
#!python -m pip install -U textblob

In [ ]:
#from textblob import TextBlob
import pandas as pd

In [ ]:
df = pd.read_csv('../input/excelrp36/train (2).csv')
df1 = pd.read_csv('../input/excelrp36/test (2).csv')

In [ ]:
import string
df['Review'] = df['Review'].apply(lambda x:x.lower())

In [ ]:
df['Review'].str.replace('[{}]'.format(string.punctuation), '') #punctuation removed

In [ ]:
print('Number of rows and columns in the train data set:',df.shape)

In [ ]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [ ]:
# Natural Language Tool Kit 
import nltk
import re  
   
# to remove stopword 
from nltk.corpus import stopwords 
  
# for Stemming propose  
from nltk.stem.porter import PorterStemmer 
  
# Initialize empty array 
# to append clean text  
corpus = []  
  
# 6148 (reviews) rows to clean 
for i in range(0, 14343):  
      
    # column : "Review", row ith 
    review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])  
      
    # convert all cases to lower cases 
    review = review.lower()  
      
    # split to array(default delimiter is " ") 
    review = review.split()  
      
    # creating PorterStemmer object to 
    # take main stem of each word 
    ps = PorterStemmer()  
      
    # loop for stemming each word 
    # in string array at ith row     
    review = [ps.stem(word) for word in review  for word in review if word not in stop_words]  
                  
    # rejoin all string array elements 
    # to create back into a string 
    review = ' '.join(review)   
      
    # append each string to create 
    # array of clean text  
    corpus.append(review) 

In [ ]:
df.head(10)

In [ ]:
polarity=[] # list which will contain the polarity of the comments
subjectivity=[] # list which will contain the subjectivity of the comments
for i in df['Review'].values:
    try:
        analysis =TextBlob(i)
        polarity.append(analysis.sentiment.polarity)
        subjectivity.append(analysis.sentiment.subjectivity)
        
    except:
        polarity.append(0)
        subjectivity.append(0)

In [ ]:
#Adding the Sentiment and Polarity column to the data
df['polarity']=polarity
df['subjectivity']=subjectivity

In [ ]:
#Displaying the NEGATIVE comments
df[['ID','Review','Rating','polarity','subjectivity']][df.polarity<0].head(10)

In [ ]:
#Displaying the POSITIVE comments
df[['ID','Review','Rating','polarity','subjectivity']][df.polarity>0].head(10)

In [ ]:
#Displaying the NEUTRAL comments
df[['ID','Review','Rating','polarity','subjectivity']][df.polarity==0].head(10)

In [ ]:
#Displaying highly subjective reviews
df[['ID','Review','Rating','polarity','subjectivity']][df.subjectivity>0.8].head(10)

In [ ]:
#Displaying highly positive reviews
df[['ID','Review','Rating','polarity','subjectivity']][df.polarity>0.8].head(10)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(df["Review"][df.polarity>0.8])

In [ ]:
#Displaying highly negative reviews
df[['ID','Review','Rating','polarity','subjectivity']][df.polarity<-0.4].head(10)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'black',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()

# print wordcloud
show_wordcloud(df["Review"][df.polarity<-0.4])

In [ ]:
#Distribution of Polarity
df.polarity.hist(bins=50)

In [ ]:
#Distribution of Subjectivity
df.subjectivity.hist(bins=50)

In [ ]:
df.head()

In [ ]:
#Converting the polarity values from continuous to categorical
df['polarity'][df.polarity==0]= 0 
df['polarity'][df.polarity > 0]= 1
df['polarity'][df.polarity < 0]= -1

In [ ]:
df.polarity.value_counts().plot.bar()
df.polarity.value_counts()

In [ ]:
#Reviews that have lowest polarity (most negative sentiment) but with a 5-star:
df[(df.Rating == 5) & (df.polarity == -1)].head(5)

In [ ]:
#Reviews that have the highest polarity (most positive sentiment) but with a 1-star:
df[(df.Rating == 1) & (df.polarity == 1)].head(5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvector = CountVectorizer(stop_words=set(stopwords.words("english")),min_df = 0.0, max_df = 1.0, ngram_range=(1,2))
cvector.fit(df.Review)

In [ ]:
len(cvector.get_feature_names())

In [ ]:
neu_matrix = cvector.transform(df[df.polarity == 0].Review)

In [ ]:
neg_matrix = cvector.transform(df[df.polarity < 0].Review)

In [ ]:
pos_matrix = cvector.transform(df[df.polarity > 0].Review)

In [ ]:
neg_words = neg_matrix.sum(axis=0)
neg_words_freq = [(word, neg_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
neg_tf = pd.DataFrame(list(sorted(neg_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','negative'])
neg_tf_df = neg_tf.set_index('Terms')
neg_tf_df.head()

In [ ]:
neu_words = neu_matrix.sum(axis=0)
neu_words_freq = [(word, neu_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
neu_words_tf = pd.DataFrame(list(sorted(neu_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','neutral'])
neu_words_tf_df = neu_words_tf.set_index('Terms')
neu_words_tf_df.head()

In [ ]:
pos_words = pos_matrix.sum(axis=0)
pos_words_freq = [(word, pos_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
pos_words_tf = pd.DataFrame(list(sorted(pos_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','positive'])
pos_words_tf_df = pos_words_tf.set_index('Terms')
pos_words_tf_df.head()

In [ ]:
term_freq_df = pd.concat([neg_tf_df,neu_words_tf_df,pos_words_tf_df],axis=1)

In [ ]:
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['neutral'] +  term_freq_df['positive'] 
term_freq_df.sort_values(by='total', ascending=False).head(10)

In [ ]:
from pylab import *
counts = term_freq_df.total
tokens = term_freq_df.index
ranks = arange(1, len(counts)+1)
indices = argsort(-counts)
frequencies = counts[indices]
plt.figure(figsize=(8,6))
plt.ylim(1,10**6)
plt.xlim(1,10**6)
loglog(ranks, frequencies, marker=".")
plt.plot([1,frequencies[0]],[frequencies[0],1],color='r')
title("Zipf plot for phrases tokens")
xlabel("Frequency rank of token")
ylabel("Absolute frequency of token")
grid(True)
for n in list(logspace(-0.5, log10(len(counts)-2), 10).astype(int)):
    dummy = text(ranks[n], frequencies[n], " " + tokens[indices[n]], 
                 verticalalignment="bottom", horizontalalignment="left")

In [ ]:
y_pos = np.arange(50)
plt.figure(figsize=(12,10))
plt.bar(y_pos, term_freq_df.sort_values(by='negative', ascending=False)['negative'][:50], align='center', alpha=0.5)
plt.xticks(y_pos, term_freq_df.sort_values(by='negative', ascending=False)['negative'][:50].index,rotation='vertical')
plt.ylabel('Frequency')
plt.xlabel('Top 50 negative tokens')
plt.title('Top 50 tokens in negative reviews')

In [ ]:
y_pos = np.arange(50)
plt.figure(figsize=(12,10))
plt.bar(y_pos, term_freq_df.sort_values(by='positive', ascending=False)['positive'][:50], align='center', alpha=0.5)
plt.xticks(y_pos, term_freq_df.sort_values(by='positive', ascending=False)['positive'][:50].index,rotation='vertical')
plt.ylabel('Frequency')
plt.xlabel('Top 50 positive tokens')
plt.title('Top 50 tokens in positive reviews')

In [ ]:
#Feature Engineering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
phrase = np.array(df['Review'])

In [ ]:
## Build Bag-Of-Words on train phrases
cv = CountVectorizer(stop_words='english',max_features=10000)
cv_train_features = cv.fit_transform(Review_train)